## 1. Basic idea

The algorithm is supposed to generate a list of rules based on PRISM algorithm discussed during the class, and should be able to work with both numerical and categorical data with no empty cells. This project is implemented based on Professor 	Marina Barsky's implementation plan and some code is borrowed from her. I also got direct help from her with regard to implementing the learn_one_rule algorithm.


## 2. Implementation notes

First, we need a datastructure to store each rule. The data structure is borrowed from Professor Marina Barsky


Each Rule consists of antecedent (Left Hand Side) and consequent (Right Hand Side). The LHS includes multiple conditions joined with AND, and RHS is a class label. The Rule also needs to store its accuracy and coverage.

In [1]:
class Rule:
    def __init__(self, class_label):
        self.conditions = []  # list of conditions
        self.class_label = class_label  # rule class
        self.accuracy = 0
        self.coverage = 0

    def addCondition(self, condition):
        self.conditions.append(condition)

    def setParams(self, accuracy, coverage):
        self.accuracy = accuracy
        self.coverage = coverage
    
    # Human-readable printing of this Rule
    def __repr__(self):
        return "If {} then {}. Coverage:{}, accuracy: {}".format(self.conditions, self.class_label,
                                                                 self.coverage, self.accuracy)

The list of conditions contains several objects of class _Condition_. 

Each condition includes the _attribute name_ and the _value_. 

If the _value_ is numeric, then the condition also includes an additional field `true_false` which means the following: 
- *if true_false == True then values are >= value* 
- *if true_false == False then values are < value*
- If *true_false is None*, then this condition is simply of form *categorical attribute = value*.

In [2]:
class Condition:
    def __init__(self, attribute, value, true_false = None):
        self.attribute = attribute
        self.value = value
        self.true_false = true_false

    def __repr__(self):
        if self.true_false is None:
            return "{}={}".format(self.attribute, self.value)
        else:
            return "{}>={}:{}".format(self.attribute, self.value, self.true_false)

Next comes the `learn_one_rule` algorithm. The required parameters are the names of the columns, the current subset of data, and the class label (RHS of the rule we are trying to learn). The optional parameters are thresholds `min_coverage` and `min_accuracy`. For normal datasets we can set the coverage value to 30, for example, to prevent creating unreliable rules with insignificant support from data.

The algorithm returns a new rule and maybe a subset of data covered by the Rule or the remaining data (not covered by the rule).    

Please note that the algorithm I implemented is prioritizing accuracy, which means that for a 1.0 accuracy rule with 30 coverage, and a 0.9 accuracy rule with 30000 coverage, the first rule would be selected. 

The general idea of the implementation is that: 
1. Create a subset of the original data set that only contains the class label we are interested in 
2. Find all possible conditions, and for each condition, check if it is int(numerical) or not int(categorical data).
3. If it is numerical data, get all possible values and sort the unique values from small to large. For all possible values, test the conditions as a. greater or equal to the value, or b, smaller than the value. Find the greater accuracy. The accuracy is compared with the current best accuracy and coverage.
4. If it is categorical, iterate through all possible values to see what is the accuracy and coverage for each category.
5. Check the current best result: 
- if I reach coverage 1, that means I can no longer improve and since every new condition would at least keep/reduce my covered subset, I don't want to continue. Thus, it is the best option. I add the current condition to my rule
- else if I tested all possible conditions, I want to see if I reached a final accpetable result or not. If so, I return the rule with all conditions in it.
- else if I reached a coverage smaller than min coverage, that means I can no longer subdivide, and have to stop. I check if it is good enough and if so I return the rule without current condition, as the current condition's coverage is too small.
- else I have to continue, and I remove current attribute from the possible attribute list, and add the condition to the rule, and continue with the iterative loop 

In [3]:
import pandas as pd
import numpy as np
def learn_one_rule(column, data, class_label, min_coverage=0, min_accuracy=0.6):
    if len(data)==0:
        return None
    covered_subset = data.copy()
    data2 = data.copy()
    current_rule = Rule(class_label)
    done = False
    # filter out data with right labels
    classheader = data2.columns[-1]
    current_data = data2[data2[classheader] == class_label]
    # make sure I am not operating the wrong set of data
    columns = column.copy()
    columns.pop()
    
    #get the best possible option
    while not done:
        #reset
        current_accuracy = 0
        current_coverage = 0
        current_condition = Condition(None,None)
        index = 0
        #go through all possible attributes
        for i in range(len(columns)):

            current_attribute = columns[i]        
        
             #check if it is numerical, get unique values, for each value, there is two rules: greater or equal to, or less than
            possible_values = current_data[current_attribute].unique().tolist()
            if isinstance(possible_values[0], int) or isinstance(possible_values[0],float):
                possible_values.sort()
             #test every possible value for the current attribute
            for value in possible_values:
                #if dealing with numerical values
                if isinstance(value, int) or isinstance(value,float):
                    #get two sets of accuracy and coverage
                    mycolumn1 = current_data[current_attribute]
                    correct1 = mycolumn1[mycolumn1 >= value].count()
                    mycolumn2 = data2[current_attribute]
                    coverage1 = mycolumn2[mycolumn2 >= value].count()
                    # this is to avoid 0/0 scenario
                    if coverage1 == 0:
                        accuracy1 = 0
                    else:
                        accuracy1 = correct1/coverage1
                    mycolumn3 = current_data[current_attribute]
                    correct = mycolumn3[mycolumn3 <value].count()
                    column4 = data2[current_attribute]
                    coverage = column4[column4 < value].count()
                    if coverage == 0:
                        accuracy = 0
                    else:
                        accuracy = correct/coverage
                    #just here to avoid syntax problems, also set flag for later use
                    GreaterThan = False
                    # get the best among the two
                    if accuracy1 > accuracy:
                        GreaterThan = True
                        accuracy = accuracy1
                        coverage = coverage1
                    elif accuracy1 == accuracy:
                        if coverage1 > coverage: 
                            GreaterThan = True
                            coverage = coverage1
                    Int = True
                    # if the result is good enough to be recorded, record as condition, and update the 
                    # accuracy/coverage/covered set
                    if coverage >= min_coverage:
                        if accuracy > current_accuracy:
                            index = i
                            current_coverage = coverage
                            current_accuracy = accuracy
                            current_condition= Condition(current_attribute, value, GreaterThan) 
                            if GreaterThan: 
                                covered_subset = data2[data2[current_attribute] >= value] 
                            else:
                                covered_subset = data2[data2[current_attribute] < value] 
                        # if accuracy is the same, compare coverage
                        elif accuracy == current_accuracy and coverage > current_coverage:
                            index = i
                            current_coverage = coverage
                            current_accuracy = accuracy
                            current_condition= Condition(current_attribute, value,GreaterThan) 
                            if GreaterThan: 
                                covered_subset = data2[data2[current_attribute] >= value] 
                            else:
                                covered_subset = data2[data2[current_attribute] < value]
                else:
                #compute accuracy
                    correct = current_data[current_attribute].value_counts()[value]
                    coverage = data2[current_attribute].value_counts()[value]
                    accuracy = correct/coverage
                    #choose the best option based on accuracy
                    if coverage >= min_coverage:
                        if accuracy > current_accuracy:
                            current_coverage = coverage
                            current_accuracy = accuracy
                            current_condition= Condition(current_attribute, value) 
                            covered_subset = data2[data2[current_attribute] == value] 
                            index = i
                        # if accuracy is the same, compare coverage
                        elif accuracy == current_accuracy and coverage > current_coverage:
                            current_coverage = coverage
                            current_accuracy = accuracy
                            current_condition= Condition(current_attribute, value) 
                            covered_subset = data2[data2[current_attribute] == value] 
                            index = i
        #if reached to the end accuracy = 1.0, add to rule and terminate
        if current_accuracy == 1.0 and current_coverage > min_coverage:
            done = True
            current_rule.addCondition(current_condition)
            current_rule.accuracy = current_accuracy
            current_rule.coverage = current_coverage
        #if reached to the end that is acceptable, add to rule and terminate
        elif len(columns) == 0: 
            done = True  
            if current_rule.accuracy <= min_accuracy or current_rule.coverage < min_coverage:
                return (None, None)

        #if no rule possible, return none, else, we reached an end and is done
        elif current_coverage < min_coverage:

            if current_rule.accuracy <= min_accuracy or current_rule.coverage < min_coverage:
                return (None, None)
            done = True                
        #default: not done yet, continue
        else: 
            #update possible attributes
            columns.pop(index)
            current_rule.addCondition(current_condition)
            data2 = covered_subset    
            current_data = data2[data2[classheader] == class_label]
            current_rule.accuracy = current_accuracy
            current_rule.coverage = current_coverage
            #reset
    return (current_rule, covered_subset)

Finally, the main algorithm `learn_rules` takes as parameters list of columns, with the last column representing the class label, and the original data in form of pandas dataframe. Two optional threshold parameters `min_coverage` and `min_accuracy` set up the conditions of rule's validity for a given dataset.

In order to cover the PRISM algorithm, I also added a small part that would run the learn_one_rule algorithm repeatedly to generate the best possible result. Unfortunately, that means the time is at least doubled and I am unable to improve on that.

In [4]:
import pandas as pd
import numpy as np

def learn_rules (columns, data, classes=None, 
                 min_coverage = 30, min_accuracy = 0.6):
    # List of final rules
    rules = []
    
    # If list of classes of interest is not provided - it is extracted from the last column of data
    if classes is not None:
        class_labels = classes
    else:
        class_labels = data[columns[-1]].unique().tolist()

    current_data = data.copy()
    
    # This follows the logic of the original PRISM algorithm
    # It processes each class in turn. 
    for class_label in class_labels:
        done = False
        while len(current_data) > min_coverage and not done:
            # Learn one rule 
            rule, subset = learn_one_rule(columns, current_data, class_label, min_coverage, min_accuracy)
            
            # If the best rule does not pass the coverage threshold - we are done with this class
            if rule is None:
                break
            copylabel = class_labels.copy()
            copylabel.remove(class_label)
            for mylabel in copylabel:
                myrule, mysubset = learn_one_rule(columns, current_data, mylabel, min_coverage, min_accuracy)
                if myrule is not None and (myrule.accuracy > rule.accuracy or myrule.coverage > rule.coverage):
                    rule, subset = myrule, mysubset
            # If we get the rule with accuracy and coverage above threshold
            
            if rule.accuracy >= min_accuracy:
                rules.append(rule)
                #try get the best result.
                for id in subset.index:
                    current_data.drop(index = id, inplace = True)
                    current_data = current_data.dropna()
                   
            else:
                done = True 

                    
    return rules

## 3. Correctness test  (Borrowed from Professor Marina Barsky)

Test your algorithm on the original dataset from the PRISM paper.

The dataset was downloaded from [here](https://archive.ics.uci.edu/ml/datasets/Lenses). The CSV version is included in this repository.

**Attribute Information**:

3 Classes:
- __1__ : the patient should be fitted with __hard__ contact lenses,
- __2__ : the patient should be fitted with __soft__ contact lenses,
- __3__ : the patient should __not__ be fitted with contact lenses.


Attributes:
1. age of the patient: (1) young, (2) pre-presbyopic, (3) presbyopic
2. spectacle prescription:  (1) myope, (2) hypermetrope
3. astigmatic:     (1) no, (2) yes
4. tear production rate:  (1) reduced, (2) normal

Presbyopia is physiological insufficiency of accommodation associated with the aging of the eye that results in progressively worsening ability to focus clearly on close objects. So "age=presbiopic" means old.

Hypermetropia: far-sightedness, also known as long-sightedness - cannot see close.
Myopia: nearsightedness - cannot see at distance.

In [5]:
import pandas as pd
import numpy as np

data_file = "contact_lenses.csv"
data = pd.read_csv(data_file, index_col=['id'])
data.columns

Index(['age', 'spectacles', 'astigmatism', 'tear production rate',
       'lenses type'],
      dtype='object')

We can replace numbers with actual values - for clarity.

In [6]:
# classes
conditions = [ data['lenses type'].eq(1), data['lenses type'].eq(2), data['lenses type'].eq(3)]
choices = ["hard","soft","none"]

data['lenses type'] = np.select(conditions, choices)

# age groups
conditions = [ data['age'].eq(1), data['age'].eq(2), data['age'].eq(3)]
choices = ["young","medium","old"]

data['age'] = np.select(conditions, choices)

# spectacles
conditions = [ data['spectacles'].eq(1), data['spectacles'].eq(2)]
choices = ["nearsighted","farsighted"]

data['spectacles'] = np.select(conditions, choices)

# astigmatism
conditions = [ data['astigmatism'].eq(1), data['astigmatism'].eq(2)]
choices = ["no","yes"]

data['astigmatism'] = np.select(conditions, choices)

# tear production rate
conditions = [ data['tear production rate'].eq(1), data['tear production rate'].eq(2)]
choices = ["reduced","normal"]

data['tear production rate'] = np.select(conditions, choices)
print(data)

       age   spectacles astigmatism tear production rate lenses type
id                                                                  
1    young  nearsighted          no              reduced        none
2    young  nearsighted          no               normal        soft
3    young  nearsighted         yes              reduced        none
4    young  nearsighted         yes               normal        hard
5    young   farsighted          no              reduced        none
6    young   farsighted          no               normal        soft
7    young   farsighted         yes              reduced        none
8    young   farsighted         yes               normal        hard
9   medium  nearsighted          no              reduced        none
10  medium  nearsighted          no               normal        soft
11  medium  nearsighted         yes              reduced        none
12  medium  nearsighted         yes               normal        hard
13  medium   farsighted          n

The test (do not run it before you finished the implementation of the rule learning algorithm):

In [7]:

column_list = data.columns.to_numpy().tolist()

rules = learn_rules (column_list, data, None, 1, 0.95)
for rule in rules[:20]:
    print(rule)

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0
If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0
If [astigmatism=yes, spectacles=nearsighted] then hard. Coverage:3, accuracy: 1.0
If [age=old] then none. Coverage:2, accuracy: 1.0
If [spectacles=nearsighted] then soft. Coverage:2, accuracy: 1.0
If [age=medium, spectacles=farsighted, astigmatism=yes, tear production rate=normal] then none. Coverage:1, accuracy: 1.0


After manual evaluationi the method seems to work with a better result from professor's method, meaning that it is more accuracy in terms of the PRISM algorithm.

Marina Barsky's results are given below for comparison:

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0

If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0

If [astigmatism=no, age=young] then soft. Coverage:1, accuracy: 1.0

If [astigmatism=no, age=medium] then soft. Coverage:1, accuracy: 1.0

If [age=young] then hard. Coverage:2, accuracy: 1.0

If [spectacles=nearsighted, astigmatism=yes] then hard. Coverage:2, accuracy: 1.0

Copyright &copy; 2020 Marina Barsky. All rights reserved.